The IceCube Neutrino Observatory is composed of many light detectors, called "DOMs" (digital optical modules), buried two kilometers deep below the surface of the South Pole. 

These light detectors are arranged in a roughly three dimensional grid: there are many "strings" hanging vertically downwards in the ice, and on each string there are many DOMs: 

When a high energy charged particle passes through the ice, it produces what is called Cherenkov light, which can then be detected by the many DOMs.

The locations of the DOMs that see light, the times at which they saw light, and the amount of light they see all communicate information about this charged particle.

As physicists, what we want to do is work backwards: if what we have is this information about the light seen by the detector, what can we figure out about the original particle? 